In [ ]:
!pip install transformers
!pip install -U PyPDF2
!pip install python-docx
!pip install tokenizers
!pip install torch
!pip install python-docx
!pip install transformers[torch]>=5.0.0
!pip install accelerate>=0.21.0

from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from google.colab import drive
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

drive.mount('/content/drive')

# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if filename.endswith(".pdf"):
                combined_text += read_pdf(file_path)
            elif filename.endswith(".docx"):
                combined_text += read_word(file_path)
            elif filename.endswith(".txt"):
                combined_text += read_txt(file_path)
        except UnicodeDecodeError:
            print(f"Error reading file: {file_path}")
    return combined_text

def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)

    model = GPT2LMHeadModel.from_pretrained(model_name)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model()

# Read documents from the directory
train_directory1 = '/content/drive/MyDrive/LLM_MODEL/Full_Text'
text_data1 = read_documents_from_directory(train_directory1)
text_data1 = re.sub(r'\n+', '\n', text_data1).strip()  # Remove excess newline characters

train_directory2 = '/content/drive/MyDrive/LLM_MODEL/Q_and_A'
text_data2 = read_documents_from_directory(train_directory2)
text_data2 = re.sub(r'\n+', '\n', text_data2).strip()  # Remove excess newline characters

with open("/content/drive/MyDrive/LLM_MODEL/Full_Text_Train/train1.txt", "w") as f:
    f.write(text_data1)

with open("/content/drive/MyDrive/LLM_MODEL/Q_and_A_Train/train2.txt", "w") as f:
    f.write(text_data2)

train_file_path1 = "/content/drive/MyDrive/LLM_MODEL/Full_Text_Train/train1.txt"
train_file_path2 = "/content/drive/MyDrive/LLM_MODEL/Q_and_A_Train/train2.txt"
model_name = 'gpt2'
output_dir1 = '/content/drive/MyDrive/LLM_MODEL/Full_Text_customise'
output_dir2 = '/content/drive/MyDrive/LLM_MODEL/Q_and_A_customise'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000

# Training on Full_Text
train(
    train_file_path=train_file_path1,
    model_name=model_name,
    output_dir=output_dir1,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

# Training on Q_and_A
train(
    train_file_path=train_file_path2,
    model_name=model_name,
    output_dir=output_dir2,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Step,Training Loss


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss


In [ ]:
!pip install transformers>=4.11.0


from transformers import PreTrainedTokenizerFast

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [ ]:

model1_path = "/content/drive/MyDrive/LLM_MODEL/Full_Text_customise"
sequence1 = "What is modern agriculture?"
max_len = 50
generate_text(model1_path, sequence1, max_len)

What is modern agriculture?
Modern agriculture emphasizes precision, productivity, and sustainability. It emphasizes precision, productivity, and sustainability. Through rigorous techniques and practices, modern agriculture offers numerous benefits. It reduces waste, increases productivity, and reduces waste and environmental impact


In [ ]:
model2_path = "/content/drive/MyDrive/LLM_MODEL/Q_and_A_customise"
sequence2 = "[Q] What is modern agriculture?"
max_len = 50
generate_text(model1_path, sequence1, max_len)

What is modern agriculture?
Modern agriculture emphasizes precision and efficiency, sustainability, and biodiversity for its workers and consumers. Technological advancements have enabled farmers to cover large areas in a fraction of the time it would take with conventional methods, reducing waste and environmental
